In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)


train_set = torchvision.datasets.FashionMNIST(
    './FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]) # converting the images ToTensor
)


In [2]:
class NetWork(nn.Module):
    
    def __init__(self):
        super(NetWork, self).__init__()
        
        #### conv layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)  
        
        #### Linear layers
        self.fc1 = nn.Linear(in_features= 12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120 , out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
            
        
    def forward(self, t):
        t = t #input layer
        
        #### hidden conv1 layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #### hidden conv2 layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #### hidden Linear_1 layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        #### hidden Linear_2 layer
        t = self.fc2(t)
        t = F.relu(t)
        
        #### hidden Linear_out layer
        t = self.out(t)
        
        return t

In [3]:
network = NetWork()

In [4]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [5]:
#### will calu the num of correct prediction out of the batch_size
def get_num_corrected(pred, labels):
    return pred.argmax(dim=1).eq(labels).sum().item()

# Tensorboar writer

In [6]:
from torch.utils.tensorboard import SummaryWriter

tb = SummaryWriter()

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
tb.add_image('tensorboardImages/', grid)
tb.add_graph(network, images)

for epoch in range(5):
    
    total_loss = 0
    total_corr = 0
    
    for batch in train_loader:
        images, labels = batch
        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_corr += get_num_corrected(preds, labels)
        
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Correct_num', total_corr, epoch)
    tb.add_scalar('Accuracy', total_corr / len(train_set))
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)

    print(f"epoch: {epoch}, total correct pred: {total_corr}, total Error pred: {total_loss}")


acc = 100* total_corr / len(train_set)
print(acc)

tb.close()


epoch: 0, total correct pred: 47771, total Error pred: 328.2219439595938
epoch: 1, total correct pred: 51420, total Error pred: 230.23466849327087
epoch: 2, total correct pred: 52152, total Error pred: 213.44688542187214
epoch: 3, total correct pred: 52583, total Error pred: 201.14260031282902
epoch: 4, total correct pred: 52733, total Error pred: 196.25219748914242
87.88833333333334
